# Setup

In [1]:
import os
import pandas as pd

In [2]:
# data paths:
ORIG_SPLIT_METADATA_FILES = {
    "unbalanced_train": "/proj/systewar/datasets/audioset_music_mood/metadata_unbalanced_train.csv",
    "balanced_train": "/proj/systewar/datasets/audioset_music_mood/metadata_balanced_train.csv",
    "eval": "/proj/systewar/datasets/audioset_music_mood/metadata_eval.csv"
}

In [3]:
# script options:
N_class_val = 150     # number of samples per class in validation set
N_class_test = 150     # number of samples per class in test set
new_split_metadata_dir = "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files"
new_split_metadata_files = {
    "train": "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files/metadata_train.csv",
    "val": "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files/metadata_val.csv",
    "test": "/proj/systewar/datasets/audioset_music_mood/new_split_metadata_files/metadata_test.csv"
}

# Get Audio Files

In [4]:
# load original split metadata files:
orig_split_metadata_dfs = {}
for subset, file_path in ORIG_SPLIT_METADATA_FILES.items():
    print("Loading {} set labels...".format(subset))
    orig_split_metadata_dfs[subset] = pd.read_csv(file_path)

# concatenate original split labels into a single dataframe:
all_metadata = pd.concat(orig_split_metadata_dfs.values(), axis="index")
all_metadata = all_metadata.reset_index(drop=True)
print()
print(all_metadata.info())

Loading unbalanced_train set labels...
Loading balanced_train set labels...
Loading eval set labels...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13713 entries, 0 to 13712
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     13713 non-null  object
 1   file_name       13713 non-null  object
 2   length_samples  13713 non-null  int64 
 3   label           13713 non-null  object
dtypes: int64(1), object(3)
memory usage: 428.7+ KB
None


In [5]:
# print validation and test sizes:
n_classes = len(all_metadata["label"].unique())
val_set_size = N_class_val * n_classes
print("Validation set size: {} samples = {:.2f}% of entire dataset.".format(val_set_size, 100 * (val_set_size / all_metadata.shape[0])))
test_set_size = N_class_test * n_classes
print("Test set size: {} samples = {:.2f}% of entire dataset.".format(test_set_size, 100 * (test_set_size / all_metadata.shape[0])))

Validation set size: 1050 samples = 7.66% of entire dataset.
Test set size: 1050 samples = 7.66% of entire dataset.


In [6]:
# get label counts:
label_counts = all_metadata["label"].value_counts()
for label in all_metadata["label"].value_counts().index:
    print("Number of {} clips: {}".format(label, label_counts[label]))

Number of Exciting music clips: 4576
Number of Tender music clips: 3358
Number of Scary music clips: 1401
Number of Sad music clips: 1385
Number of Happy music clips: 1152
Number of Angry music clips: 946
Number of Funny music clips: 895


# Split Dataset

In [7]:
# construct stratified test set by randomly sampling from each class:
metadata_test_groups = all_metadata.groupby(by="label", axis="index")
metadata_test = metadata_test_groups.sample(n=N_class_test, random_state=42)
print(metadata_test.info())

# sanity check:
for count in metadata_test["label"].value_counts():
    assert count == N_class_test, "Error with creating stratified test set."

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050 entries, 5169 to 8349
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     1050 non-null   object
 1   file_name       1050 non-null   object
 2   length_samples  1050 non-null   int64 
 3   label           1050 non-null   object
dtypes: int64(1), object(3)
memory usage: 41.0+ KB
None


In [8]:
# remove test set:
metadata_train_val = all_metadata.drop(index=list(metadata_test.index))

# construct stratified validation set by randomly sampling from each class:
metadata_val_groups = metadata_train_val.groupby(by="label", axis="index")
metadata_val = metadata_val_groups.sample(n=N_class_val, random_state=42)
print(metadata_val.info())

# sanity check:
for count in metadata_val["label"].value_counts():
    assert count == N_class_val, "Error with creating stratified validation set."

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050 entries, 12807 to 354
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     1050 non-null   object
 1   file_name       1050 non-null   object
 2   length_samples  1050 non-null   int64 
 3   label           1050 non-null   object
dtypes: int64(1), object(3)
memory usage: 41.0+ KB
None


In [9]:
# construct training set:
metadata_train = metadata_train_val.drop(index=list(metadata_val.index))
print(metadata_train.info())
print()
print(metadata_train["label"].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613 entries, 0 to 13710
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   orig_subset     11613 non-null  object
 1   file_name       11613 non-null  object
 2   length_samples  11613 non-null  int64 
 3   label           11613 non-null  object
dtypes: int64(1), object(3)
memory usage: 453.6+ KB
None

Exciting music    4276
Tender music      3058
Scary music       1101
Sad music         1085
Happy music        852
Angry music        646
Funny music        595
Name: label, dtype: int64


In [11]:
# check that all subsets are disjoint:
metadata_subsets = [metadata_train, metadata_val, metadata_test]
subset_names = list(new_split_metadata_files.keys())
for subset_1, name_1 in zip(metadata_subsets, subset_names):
    for subset_2, name_2 in zip(metadata_subsets, subset_names):
        if name_1 != name_2:
            assert set(subset_1.index).isdisjoint(set(subset_2.index)), "{} and {} are not disjoint".format(name_1, name_2)

In [12]:
# reset indices:
metadata_train = metadata_train.reset_index(drop=True)
metadata_val = metadata_val.reset_index(drop=True)
metadata_test = metadata_test.reset_index(drop=True)

# sanity checks:
assert all_metadata.shape[0] == metadata_train.shape[0] + metadata_val.shape[0] + metadata_test.shape[0], "Subset set sizes don't add up."
# check that all subsets are disjoint:
metadata_subsets = [metadata_train, metadata_val, metadata_test]
subset_names = list(new_split_metadata_files.keys())
for subset_1, name_1 in zip(metadata_subsets, subset_names):
    for subset_2, name_2 in zip(metadata_subsets, subset_names):
        if name_1 != name_2:
            assert set(subset_1["file_name"].tolist()).isdisjoint(set(subset_2["file_name"].tolist())), "{} and {} are not disjoint".format(name_1, name_2)
# more sanity checks:
class_counts_all = all_metadata["label"].value_counts()
class_counts_train = metadata_train["label"].value_counts()
class_counts_val = metadata_val["label"].value_counts()
class_counts_test = metadata_test["label"].value_counts()
for class_label in metadata_test_groups.groups.keys():
    assert class_counts_all[class_label] == class_counts_train[class_label] + class_counts_val[class_label] + class_counts_test[class_label], "Error with splitting dataset."

# save to file:
metadata_train.to_csv(new_split_metadata_files["train"], index=False)
metadata_val.to_csv(new_split_metadata_files["val"], index=False)
metadata_test.to_csv(new_split_metadata_files["test"], index=False)